In [1]:
from datasets import load_dataset
ds = load_dataset("apurvaga/go-browse-wa-raw",cache_dir="./")

d:\anaconda3\envs\go-browse\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import re
import os
from tqdm import tqdm
import random
import json
from datasets import load_dataset
from collections import defaultdict

d:\anaconda3\envs\go-browse\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
CACHE_DIR = './apurvaga___go-browse-wa-raw'
ds = load_dataset("apurvaga/go-browse-wa-raw", split="train", streaming=True, cache_dir=CACHE_DIR)
SAMPLE_SIZE = 500 

In [3]:
row = next(iter(ds))
print(json.dumps(row, indent=2, ensure_ascii=False, default=str))
print(list(row.keys()))

{
  "__key__": "00000-00",
  "__url__": "/data/user_data/apurvag/web_explore_data/go-browse-wa-raw/shard-000000.tar",
  "json": {
    "graph_data": {
      "domain": "map",
      "root_url": "http://ec2-3-148-123-246.us-east-2.compute.amazonaws.com:3000/#map=7/42.896/-75.108"
    },
    "node_data": {
      "node_url": "http://ec2-3-148-123-246.us-east-2.compute.amazonaws.com:3000/#map=7/42.896/-75.108"
    },
    "step_data": {
      "action": "I need to find driving directions from Central Park, New York to Times Square, New York using the Car (OSRM) option.\n\n```json\n{\n  \"thought\": \"Looking at the current page, I can see I'm on OpenStreetMap. To find directions, I need to click on the 'Find directions between two points' link which has a directions icon. I can see this element with ID 149 in the accessibility tree.\",\n  \"action\": \"click('149')\"\n}\n```",
      "misc": {
        "agent_config": {
          "char_limit": 80000,
          "demo_mode": "off",
          "model

In [8]:
success_count = 0
for i, row in enumerate(ds):
    if i >= SAMPLE_SIZE:
        break 
    json_data = row['json']
    traj_data = json_data.get('traj_data', {})
    if not traj_data.get('success', False):
        continue
    success_count += 1
print(success_count)

87


In [7]:
def parsed_element(parsed_action):
    if not parsed_action: return None, None, None
    op = parsed_action.split('(')[0].strip()

    id_match = re.search(r"['\"\(](\d+)(?:['\"\)]|$)", str(parsed_action))
    if id_match:
        return op, id_match.group(1), "id"
    
    start_quote = re.search(r"(['\"])(.*)", str(parsed_action))
    if start_quote:
        raw_text = start_quote.group(2)
        clean_text = raw_text.rstrip(')').rstrip("'").rstrip('"').rstrip('\\')
        try:
            final_text = clean_text.encode('utf-8').decode('unicode_escape')
            return op, final_text, "text"
        except:
            return op, clean_text, "text"
            
    return op, None, None

def get_element_desc(full_tree, target, match_type="id"):
    if not full_tree or not target: return None
    
    lines = full_tree.split('\n')
    
    if match_type == "id":
        target_token = f"[{target}]"
        for line in lines:
            if line.strip().startswith(target_token):
                return line.strip()
                
    elif match_type == "text":
        for line in lines:
            if target in line:
                return line.strip()
                
    return None

test_action = "click(\"\\\\ue622 Sports & Outdoors\\\\"
parsed_element(test_action)



('click', '\\ue622 Sports & Outdoors', 'text')

In [8]:
def parsed_from_jsonl(row) -> dict:
    json_data = row['json']
    traj_data = json_data.get('traj_data', {})    
    traj_num = traj_data.get('traj_num',None)
    goal = traj_data.get('goal',None)
    domain = json_data.get('graph_data',{}).get('domain',None)
    thought = json_data.get('step_data',{}).get('thought',None)
    parsed_action = json_data.get('step_data',{}).get('parsed_action',None)
    axtree_txt = json_data.get('step_data',{}).get('obs',{}).get('axtree_txt',None)
    axtree_visible_only_txt = json_data.get('step_data',{}).get('obs',{}).get('axtree_visible_only_txt',None)
    step_num = json_data.get('step_data',{}).get('obs',{}).get('step_number',0)
    temp_op,temp_id,match_type = parsed_element(parsed_action=parsed_action)
    target_element_desc = get_element_desc(axtree_txt,temp_id,match_type)
    data_dict = {
            "traj_num": traj_num,
            "goal": goal,
            "domain": domain,
            "step_num":step_num,
            "thought":thought,
            "parsed_action":parsed_action,
            "target_element_desc":target_element_desc,
            "axtree_txt":axtree_txt,
            "axtree_visible_only_txt":axtree_visible_only_txt,
        }
    
    return data_dict

In [9]:
data_dict = parsed_from_jsonl(row=row)
print(json.dumps(data_dict, ensure_ascii=False, indent=4))

{
    "traj_num": 0,
    "goal": "Find driving directions from Central Park, New York to Times Square, New York using the Car (OSRM) option.",
    "domain": "map",
    "step_num": 0,
    "thought": "Looking at the current page, I can see I'm on OpenStreetMap. To find directions, I need to click on the 'Find directions between two points' link which has a directions icon. I can see this element with ID 149 in the accessibility tree.",
    "parsed_action": "click('149')",
    "target_element_desc": "[149] link 'Find directions between two points', url='http://ec2-3-148-123-246.us-east-2.compute.amazonaws.com:3000/directions'",
    "axtree_txt": "RootWebArea 'OpenStreetMap', focused, url='http://ec2-3-148-123-246.us-east-2.compute.amazonaws.com:3000/#map=7/42.896/-75.108'\n\t[44] banner ''\n\t\t[45] heading 'OpenStreetMap logo OpenStreetMap'\n\t\t\t[46] link 'OpenStreetMap logo OpenStreetMap', url='http://ec2-3-148-123-246.us-east-2.compute.amazonaws.com:3000/#map=7/42.896/-75.108'\n\t\t\

In [13]:
RANDOM_SEED = 4593
SAMPLE_SIZE = 1
shuffled_ds = ds.shuffle(seed=RANDOM_SEED)

random_sample_ds = shuffled_ds.take(SAMPLE_SIZE)

for row in random_sample_ds:
    data_dict = parsed_from_jsonl(row=row)
    print(json.dumps(data_dict, ensure_ascii=False, indent=4))

{
    "traj_num": 742,
    "goal": "Change the transportation mode to Bicycle for directions from Times Square, New York to Central Park, New York.",
    "domain": "map",
    "step_num": 0,
    "thought": "I need to change the transportation mode to Bicycle, which is currently set to Car (OSRM). To do this, I will select the Bicycle (OSRM) option from the transportation mode dropdown menu.",
    "parsed_action": "select_option('166', 'Bicycle (OSRM)')",
    "target_element_desc": "[166] combobox '' value='Car (OSRM)', hasPopup='menu', expanded=False",
    "axtree_txt": "RootWebArea 'OpenStreetMap', focused, url='http://ec2-3-148-123-246.us-east-2.compute.amazonaws.com:3000/directions'\n\t[44] banner ''\n\t\t[45] heading 'OpenStreetMap logo OpenStreetMap'\n\t\t\t[46] link 'OpenStreetMap logo OpenStreetMap', url='http://ec2-3-148-123-246.us-east-2.compute.amazonaws.com:3000/#map=7/42.896/-75.108'\n\t\t\t\t[49] image 'OpenStreetMap logo', url='http://ec2-3-148-123-246.us-east-2.compute.am

下面是分别生成micro和macro的数据集

分类标准是：
mirco要过滤掉失败的，noop的，保留相同goal中最短的traj，目的是根据goal和traj中网页状态用gpt模拟生成和后续world model给出类似的原子hint
macro要过滤掉失败的的，保留相同goal中第一步的网页状态，目的是根据goal 和 step0的网页状态用gpt模拟生成和后续world model给出类似的hint

In [ ]:
OUTPUT_FILENAME = "clean_candidates_for_gpt_micro.jsonl"

# Group by (traj_num, goal), collect all entries for each group
traj_goal_groups = defaultdict(list)  # (traj_num, goal) -> [json_str, ...]

print("Processing data...")
for i, row in enumerate(ds):
    # if i > 1000:
    #     break
    json_data = row['json']
    traj_data = json_data.get('traj_data', {})
    if not traj_data.get('success', False):
        continue
    data_dict = parsed_from_jsonl(row=row)
    action_str = data_dict.get('parsed_action', "")
    if not action_str or 'noop' in str(action_str).lower():
        continue
    
    traj_num = data_dict.get('traj_num')
    goal = data_dict.get('goal', '')
    if traj_num is not None and goal:
        json_line = json.dumps(data_dict, ensure_ascii=False)
        traj_goal_groups[(traj_num, goal)].append(json_line)

print(f"Processed {sum(len(entries) for entries in traj_goal_groups.values())} entries")
print(f"Found {len(traj_goal_groups)} unique (traj_num, goal) combinations")

# For each goal, find traj_num with minimum entries
goal_to_traj = defaultdict(list)  # goal -> [(traj_num, count), ...]
for (traj_num, goal), entries in traj_goal_groups.items():
    goal_to_traj[goal].append((traj_num, len(entries)))

# Select traj_num with minimum count for each goal
selected_traj_goals = {}
for goal, traj_list in goal_to_traj.items():
    # Find traj_num with minimum count
    min_traj = min(traj_list, key=lambda x: x[1])
    selected_traj_goals[(min_traj[0], goal)] = min_traj[1]

print(f"Selected {len(selected_traj_goals)} (traj_num, goal) combinations")

# Show goals with multiple traj_num
multi_traj = {goal: traj_list for goal, traj_list in goal_to_traj.items() if len(traj_list) > 1}
if multi_traj:
    print(f"\nGoals with multiple traj_num ({len(multi_traj)}):")
    for goal, traj_list in list(multi_traj.items())[:10]:
        selected = min(traj_list, key=lambda x: x[1])
        print(f"  {goal[:60]}... | Traj options: {traj_list} | Selected: traj_num={selected[0]} (count={selected[1]})")

# Write selected entries to file
with open(OUTPUT_FILENAME, 'w', encoding='utf-8') as f:
    total_written = 0
    for (traj_num, goal) in selected_traj_goals:
        entries = traj_goal_groups[(traj_num, goal)]
        for entry in entries:
            f.write(entry + '\n')
            total_written += 1

print(f"\nSaved {total_written} entries to {OUTPUT_FILENAME}")

Processing data...


<frozen _collections_abc>:845: RuntimeWarning: coroutine 'main' was never awaited


Processed 34289 entries
Found 9502 unique (traj_num, goal) combinations
Selected 2991 (traj_num, goal) combinations

Goals with multiple traj_num (2108):
  Find driving directions from Central Park, New York to Times... | Traj options: [(0, 5), (1, 5), (2, 5), (3, 5), (4, 7)] | Selected: traj_num=0 (count=5)
  Navigate to the History page to view changes.... | Traj options: [(15, 2), (16, 2), (17, 2), (18, 2), (19, 2)] | Selected: traj_num=15 (count=2)
  Navigate to the GPS Traces page to view available traces.... | Traj options: [(24, 2), (25, 2), (26, 2)] | Selected: traj_num=24 (count=2)
  Navigate to the User Diaries page to explore user contributi... | Traj options: [(33, 2), (34, 2), (35, 2), (36, 2)] | Selected: traj_num=33 (count=2)
  Navigate to the Communities page to find community resources... | Traj options: [(42, 2), (43, 2), (44, 2), (45, 2)] | Selected: traj_num=42 (count=2)
  Navigate to the Help page for assistance with using OpenStre... | Traj options: [(51, 2), (52,

In [ ]:
OUTPUT_FILENAME = "clean_candidates_for_gpt_macro.jsonl"

traj_goal_groups = defaultdict(list)

print("Processing data...")
for i, row in enumerate(ds):
    json_data = row['json']
    traj_data = json_data.get('traj_data', {})
    if not traj_data.get('success', False):
        continue
    data_dict = parsed_from_jsonl(row=row)
    action_str = data_dict.get('parsed_action', "")
    if not action_str or 'noop' in str(action_str).lower():
        continue
    
    traj_num = data_dict.get('traj_num')
    goal = data_dict.get('goal', '')
    if traj_num is not None and goal:
        json_line = json.dumps(data_dict, ensure_ascii=False)
        traj_goal_groups[(traj_num, goal)].append(json_line)

print(f"Processed {sum(len(entries) for entries in traj_goal_groups.values())} entries")
print(f"Found {len(traj_goal_groups)} unique (traj_num, goal) combinations")

goal_to_trajs = defaultdict(set)
for (traj_num, goal), entries in traj_goal_groups.items():
    goal_to_trajs[goal].add(traj_num)

selected_traj_goals = {}
for goal, traj_set in goal_to_trajs.items():
    min_traj_num = min(traj_set)
    selected_traj_goals[(min_traj_num, goal)] = len(traj_goal_groups[(min_traj_num, goal)])

print(f"Selected {len(selected_traj_goals)} (traj_num, goal) combinations")

multi_traj = {goal: list(traj_set) for goal, traj_set in goal_to_trajs.items() if len(traj_set) > 1}
if multi_traj:
    print(f"\nGoals with multiple traj_num ({len(multi_traj)}):")
    for goal, traj_list in list(multi_traj.items())[:10]:
        selected = min(traj_list)
        count = selected_traj_goals[(selected, goal)]
        print(f"  {goal[:60]}... | Traj options: {traj_list} | Selected: traj_num={selected} (count={count})")

with open(OUTPUT_FILENAME, 'w', encoding='utf-8') as f:
    total_written = 0
    for (traj_num, goal) in selected_traj_goals:
        entries = traj_goal_groups[(traj_num, goal)]
        
        if entries:
            f.write(entries[0] + '\n')
            total_written += 1

print(f"\nSaved {total_written} entries to {OUTPUT_FILENAME}")

Processing data...
Processed 34289 entries
Found 9502 unique (traj_num, goal) combinations
Selected 2991 (traj_num, goal) combinations

Goals with multiple traj_num (2108):
  Find driving directions from Central Park, New York to Times... | Traj options: [0, 1, 2, 3, 4] | Selected: traj_num=0 (count=5)
  Navigate to the History page to view changes.... | Traj options: [15, 16, 17, 18, 19] | Selected: traj_num=15 (count=2)
  Navigate to the GPS Traces page to view available traces.... | Traj options: [24, 25, 26] | Selected: traj_num=24 (count=2)
  Navigate to the User Diaries page to explore user contributi... | Traj options: [33, 34, 35, 36] | Selected: traj_num=33 (count=2)
  Navigate to the Communities page to find community resources... | Traj options: [42, 43, 44, 45] | Selected: traj_num=42 (count=2)
  Navigate to the Help page for assistance with using OpenStre... | Traj options: [51, 52, 53, 54, 55] | Selected: traj_num=51 (count=2)
  Find New York City on the map... | Traj opt

In [35]:
def build_inverse_folding_prompt(data):
    """
    构造提示词：让 GPT 根据【结果】反推【指令】
    """
    domain = data.get('domain', 'general')
    target = data.get('target_element_desc', 'Unknown Element')
    action_code = data.get('parsed_action', 'Unknown Action')
    original_thought = data.get('thought', "No thought provided")

    return f"""
# Context
User is browsing a web page.
- Domain: {domain}
- Target Element: {target}
- Action Taken: {action_code}

# Reference Info (From the original agent)
The agent originally thought: "{original_thought}"
(NOTE: This is a Solver's internal monologue. Use it to understand the intent, but DO NOT copy its style. Your output must be from a PROPOSER's perspective.)

# Task
You are a "Task Proposer" training a web agent. Based on the target element and action, synthesize the training labels:
1. **Instruction**: A clear, natural language command that a user would give to click/use this specific element. (e.g., "Search for Sony TV", "Click the Login button").
2. **Proposer Thought**: A reasoning step explaining *why* this specific element was chosen to fulfill the instruction. Focus on planning and grounding.

# Output Format (JSON Only)
{{
  "synthesized_instruction": "...",
  "synthesized_thought": "..."
}}
"""

In [36]:
prompt = build_inverse_folding_prompt(data_dict)
print(prompt)


# Context
User is browsing a web page.
- Domain: reddit
- Target Element: None
- Action Taken: send_msg_to_user('I have explored the technology forum thoroughly. The forum contains a variety of technology-related discussions on topics like AI development, privacy concerns, tech policy, mobile devices, and innovations. I viewed posts sorted by both popularity and recency, examined a specific post with comments, and observed the forum structure and engagement. The forum appears to be active with discussions on current technology news, though most recent posts show "No comments" yet. Users can upvote/downvote posts and participate in discussions on specific technology topics that interest them.')

# Reference Info (From the original agent)
The agent originally thought: "In my last action, I attempted to send a message to the user summarizing my exploration of the technology forum, but I received an error message indicating that the action was empty. This could be due to an issue with the

In [4]:
from openai import AsyncOpenAI
import asyncio
from tqdm.asyncio import tqdm_asyncio
INPUT_FILE = "clean_candidates_for_gpt_micro.jsonl" 
OUTPUT_FILE = "sft_training_data_micro.jsonl"
API_KEY = "sk-KQa1gQlHzwWn2S32k3uAlS2HkV8EfWaAElUAtSZT7b0icndf" 
BASE_URL = "https://turingai.plus/v1" # 如果用国内中转，请修改这里

MODEL_NAME = "gpt-3.5-turbo"
CONCURRENCY = 30  
client = AsyncOpenAI(api_key=API_KEY, base_url=BASE_URL)

In [ ]:
async def process_entry(semaphore, line):
    """处理单行数据：调用 GPT -> 解析 JSON -> 合并数据"""
    async with semaphore:
        try:
            data = json.loads(line)
            prompt = build_inverse_folding_prompt(data)
            response = await client.chat.completions.create(
                model=MODEL_NAME,
                messages=[
                    {"role": "system", "content": "You are an expert data synthesizer for Web Agents. Output valid JSON only."},
                    {"role": "user", "content": prompt}
                ],
                response_format={"type": "json_object"}, 
                temperature=0.7 
            )
            
            result_text = response.choices[0].message.content
            gpt_result = json.loads(result_text)
            data['synthesized_instruction'] = gpt_result.get('synthesized_instruction')
            data['synthesized_thought'] = gpt_result.get('synthesized_thought')
            print(data['synthesized_instruction'],data['synthesized_thought'])
            
            return json.dumps(data, ensure_ascii=False)
            
        except Exception as e:
            print(f"\n[Error] Failed to process line: {e}")
            return None

In [ ]:
async def main():
    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        lines = [line.strip() for line in f if line.strip()]
        semaphore = asyncio.Semaphore(CONCURRENCY)
        tasks = [process_entry(semaphore, line) for line in lines]
        results = await tqdm_asyncio.gather(*tasks)
        success_count = 0
        with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
            for res in results:
                if res:
                    f.write(res + "\n")
                    success_count += 1
await main()

In [4]:
with open(OUTPUT_FILE, 'r', encoding='utf-8') as f:
    first_line = f.readline()
    if first_line.strip(): 
        data = json.loads(first_line)
        print(data.keys())

dict_keys(['traj_num', 'goal', 'domain', 'step_num', 'thought', 'parsed_action', 'target_element_desc', 'axtree_txt', 'axtree_visible_only_txt', 'synthesized_instruction', 'synthesized_thought'])


最终的micro训练数据集为input domain，goal，obs-axtree，target element，label为thought和instruction

In [13]:
INPUT_FILE = "sft_training_data_micro.jsonl"
OUTPUT_FILE = "sft_final_train_dataset_micro.jsonl" 

clean_data = []

with open(INPUT_FILE, 'r', encoding='utf-8') as f:
    for line in f:
        if not line.strip(): continue
        try:
            raw = json.loads(line)
            
            entry = {
                # === Input 部分 ===
                "domain": raw.get('domain', 'general'),
                "goal": raw.get('goal', 'Unknown Goal'),
                "observation": raw.get('axtree_visible_only_txt', ''),
                "target_element": raw.get('target_element_desc', 'Unknown Target'),
                
                # === Label 部分 ===
                "thought": raw.get('synthesized_thought', ''),
                "instruction": raw.get('synthesized_instruction', '')
            }
            
            if entry['thought'] and entry['instruction'] and entry['observation']:
                clean_data.append(entry)
                
        except Exception as e:
            print(f"Skipping bad line: {e}")

print(f"Writing {len(clean_data)} entries to {OUTPUT_FILE} in JSONL format...")

with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    for entry in clean_data:
        json_line = json.dumps(entry, ensure_ascii=False)
        f.write(json_line + '\n')
    
print(f"Total entries processed and saved: {len(clean_data)}")

Writing 8862 entries to sft_final_train_dataset_micro.jsonl in JSONL format...
Total entries processed and saved: 8862


In [14]:
with open(OUTPUT_FILE, 'r', encoding='utf-8') as f:
    first_line = f.readline()
    if first_line.strip(): 
        data = json.loads(first_line)
        print(data.keys())

dict_keys(['domain', 'goal', 'observation', 'target_element', 'thought', 'instruction'])


In [5]:
def build_macro_generation_prompt(data):
    """
    构造提示词：让 GPT 根据【完美目标】生成【模糊的用户输入】(模拟真实场景)
    """
    domain = data.get('domain', 'general')
    # 这是原始数据里那个写得非常完美的 Goal
    full_goal = data.get('goal', 'Unknown Goal')

    return f"""
# Context
The user is visiting a **{domain}** website (e.g., OpenStreetMap, Amazon, Reddit).
The detailed, ground-truth goal for this session is: 
"{full_goal}"

# Task
You are a Data Synthesizer. You need to create a training pair (Input -> Output) for a Macro Agent.
1. **Simulate User Input (Vague Hint)**: 
   - Rewrite the specific goal into a short, vague, natural query (3-6 words).
   - Imagine a user typing quickly into a search bar or muttering to an AI. 
   - Use lowercase, remove filler words (please, help, I want to), and focus on entities/keywords.
   
2. **Synthesize Proposer Thought**: 
   - Write the internal reasoning of the agent *after* receiving the Vague Hint but *before* taking action.
   - The thought should explain how to interpret the vague keywords into the specific goal, based on the website domain.

# Examples
## Example 1 (Map Domain)
- Specific Goal: "Find driving directions from Central Park to Times Square using Car mode."
- **vague_user_hint**: "drive central park times square"
- **synthesized_thought**: "The user input 'drive central park times square' implies a navigation task. Since I am on a Map website, I should plan to find a route between these two locations using the 'Car' option."

## Example 2 (Shopping Domain)
- Specific Goal: "Search for a black Sony noise-canceling headphone and sort by price low to high."
- **vague_user_hint**: "sony headphones black cheap"
- **synthesized_thought**: "The user wants 'sony headphones black cheap'. I need to perform a search for 'Sony noise-canceling headphones', apply the 'Black' color filter, and then set the sort order to 'Price: Low to High' to meet the 'cheap' requirement."

# Output Format (JSON Only)
{{
  "vague_user_hint": "...",
  "synthesized_thought": "..."
}}
"""

In [6]:
prompt = build_macro_generation_prompt(data_dict)
print(prompt)

NameError: name 'data_dict' is not defined

In [6]:
INPUT_FILE = "clean_candidates_for_gpt_macro.jsonl" 
OUTPUT_FILE = "sft_final_train_dataset_macro.jsonl"

In [ ]:
async def process_entry(semaphore, line):
    """
    处理Macro数据：
    1. 调用 GPT 将完美 Goal 改写为模糊 Hint
    2. 生成规划性的 Thought
    3. 构造 Macro 训练格式
    """
    async with semaphore:
        try:
            raw_data = json.loads(line)
            prompt = build_macro_generation_prompt(raw_data)
            response = await client.chat.completions.create(
                model=MODEL_NAME,
                messages=[
                    {"role": "system", "content": "You are an expert data synthesizer. Output valid JSON only."},
                    {"role": "user", "content": prompt}
                ],
                response_format={"type": "json_object"}, 
                temperature=0.7 
            )
            result_text = response.choices[0].message.content
            gpt_result = json.loads(result_text)
            vague_hint = gpt_result.get('vague_user_hint', 'unknown hint')
            syn_thought = gpt_result.get('synthesized_thought', 'planning...')
            macro_entry = {
                "domain": raw_data.get('domain', 'general'),
                "goal": f"User Hint: {vague_hint}", 
                "observation": raw_data.get('axtree_visible_only_txt', raw_data.get('axtree_txt', '')),
                "target_element": "None (Global Planning)",
                "thought": syn_thought,
                "instruction": raw_data.get('goal') 
            }
            
            print(f"[Macro] Hint: '{vague_hint}' -> Real Goal: '{macro_entry['instruction'][:50]}...'")
            
            return json.dumps(macro_entry, ensure_ascii=False)
            
        except Exception as e:
            print(f"\n[Error] Failed to process line: {e}")
            return None

In [ ]:
async def main():
    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        lines = [line.strip() for line in f if line.strip()]
        semaphore = asyncio.Semaphore(CONCURRENCY)
        tasks = [process_entry(semaphore, line) for line in lines]
        results = await tqdm_asyncio.gather(*tasks)
        success_count = 0
        with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
            for res in results:
                if res:
                    f.write(res + "\n")
                    success_count += 1
await main()

In [9]:
with open(OUTPUT_FILE, 'r', encoding='utf-8') as f:
    first_line = f.readline()
    if first_line.strip(): 
        data = json.loads(first_line)
        print(data.keys())

dict_keys(['domain', 'goal', 'observation', 'target_element', 'thought', 'instruction'])


In [ ]:
def load_data(filepath):
    data_list = []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                data_list.append(json.loads(line))
    return data_list

def save_to_jsonl(data_list, filepath):
    with open(filepath, 'w', encoding='utf-8') as f:
        for item in data_list:
            json_record = json.dumps(item, ensure_ascii=False)
            f.write(json_record + '\n')
micro = load_data("sft_final_train_dataset_macro.jsonl")
macro = load_data("sft_final_train_dataset_micro.jsonl")
final_data = micro + macro 
random.shuffle(final_data)
split = int(len(final_data) * 0.8)
train_set = final_data[:split]
val_set = final_data[split:]

save_to_jsonl(train_set, "train.jsonl")
save_to_jsonl(val_set, "val.jsonl")

train_sample_size = int(len(train_set) * 0.2)
val_sample_size = int(len(val_set) * 0.2)
test_train_sample = random.sample(train_set, train_sample_size)
test_val_sample = random.sample(val_set, val_sample_size)
save_to_jsonl(test_train_sample, "test_train.jsonl")
save_to_jsonl(test_val_sample, "test_val.jsonl")